In [1]:
%load_ext autoreload
%autoreload 2

In [6]:
from module.SeleniumCrawler import SeleniumCrawler
import pandas as pd
import os
from module.util import send_prompt, read_document

## Indeed

In [3]:
NAME = "Indeed-Data Science"
BASE_URL = "https://ca.indeed.com/jobs?q=data+scientist&fl=&sort=date&vjk=cf4843c2196f979a"
NEW_URL_CSS = 'a[aria-label*="full details"]'
NEXT_PAGE_CSS = 'a[aria-label="Next Page"]'
CV_PATH = "YOUR CV"
OPENAI_KEY = "YOUR API KEY"
cv = read_document(CV_PATH)
task_name = NAME.replace(" ", "_")
PROMPT_TEMPLATE = f"""
Based on the CV provided below, write a cover letter tailored for the job.

In the first paragraph, Introduce Geoffrey Wong, a full-stack data scientist, and how I am qualified for the role I am applying for. 
If appropriate, make a joke or a statement that aligns with the purpose in the company could have.

In the second paragraph, Highlight your most relevant skills, experiences, and achievements that directly relate to the job requirements.
Make sure to use an example that is relevant to the company purpose. Use metrics, numbers, or tangible results to quantify the example.

In the third paragraph, try to come up with a problem the company could be facing. Link it to my past experience and explain
how my expertise could help them tackle the problem.

In general, keep the writing passionate and enthusiatic.

CV
-----------------
```{cv}```

Job Position Detial
-----------------
"""

## Glassdoor

In [24]:
NAME = "Glassdoor-Data Science"
BASE_URL = "https://www.glassdoor.com/Job/taiwan-data-scientist-jobs-SRCH_IL.0,6_IN240_KO7,21.htm"
NEW_URL_CSS = 'a[href*="/partner/jobListing.htm"]'
NEXT_PAGE_CSS = 'button[aria-label="Next"]' # Does not work, since there is no url for next page
CV_PATH = "YOUR CV"
OPENAI_KEY = "YOUR API KEY"
cv = read_document(CV_PATH)
task_name = NAME.replace(" ", "_")
PROMPT_TEMPLATE = f"""
Based on the CV provided below, write a cover letter tailored for the job.

In the first paragraph, Introduce Geoffrey Wong, a full-stack data scientist, and how I am qualified for the role I am applying for. 
If appropriate, make a joke or a statement that aligns with the purpose in the company could have.

In the second paragraph, Highlight your most relevant skills, experiences, and achievements that directly relate to the job requirements.
Make sure to use an example that is relevant to the company purpose. Use metrics, numbers, or tangible results to quantify the example.

In the third paragraph, try to come up with a problem the company could be facing. Link it to my past experience and explain
how my expertise could help them tackle the problem.

In general, keep the writing passionate and enthusiatic.

CV
-----------------
```{cv}```

Job Position Detial
-----------------
"""

In [25]:
if os.path.exists(f"history/{task_name}.csv"):
    job_dataframe = pd.read_csv(f"history/{task_name}.csv")
else:
    job_dataframe = pd.DataFrame(columns=['url', 'page_source', 'content', 'cover_letter'])

crawler = SeleniumCrawler()
while BASE_URL is not None:
    urls = crawler.get_hrefs_by_css(BASE_URL,NEW_URL_CSS)
    BASE_URL = crawler.get_href_by_css(BASE_URL,NEXT_PAGE_CSS)
    
    for job_url in urls:
        source, meta = crawler.crawl_meta(job_url)
        if (meta in job_dataframe['content'].values): continue
        cover_letter = send_prompt(PROMPT_TEMPLATE + meta, OPENAI_KEY)
        print(cover_letter)
        job_dict = {
            "url": job_url,
            "page_source": source,
            "content": meta,
            "cover_letter":  cover_letter
        }
        job_dataframe.loc[len(job_dataframe)] = job_dict
        #job_dataframe = job_dataframe.append(job_dict, ignore_index=True)
        job_dataframe.to_csv(f"history/{task_name}.csv", index = False)
        input("Proceed?")
        #break

Dear Hiring Manager,

I am writing to express my deep interest in the position of data scientist at your esteemed company. As a full-stack data scientist, I have the skills and experience to meet and exceed the job requirements. What excites me most about this role is the opportunity to use my expertise to help bring business insights that will benefit your company.

Throughout my academic and professional career, I have gained extensive experience in data analytics and machine learning frameworks such as Pytorch, Keras, and Scikit-learn, among others. I have also spent time honing my skills with big data technologies, including Google Cloud Platform, Azure Model Hosting, and the likes. As a result of this, I was able to increase productivity by 10 times with a 50% reduction in costs while extracting insights using GPT-2 sentiments, analyzing interests, and picking out keywords for Hotmob Limited.

In my previous work, I have taken an active role in automating entire data processing sy

KeyboardInterrupt: Interrupted by user

In [ ]:
crawler.exit_driver()